In [2]:
import os
import gradio as gr
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import PyPDFLoader
from elevenlabs import voices, generate, play, save
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper

eleven_api_key = "34dbe78c2a8c6642fcdc27dbb44c1929"

In [22]:
def podify(topic, style, num_minutes, voice):


    num_words = num_minutes*135 # Approximately 135 words per minute with TTS voice (refine this)

    audio_voice = voice.split()[0]
    summary_lang = "English"
    if voice.split()[1] == "(SWE)":
        summary_lang = "Swedish"
    if voice.split()[1] == "(IT)":
        summary_lang = "Italian"

    wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    text = wikipedia.run(topic)    

    return "audiofiles/news_eli5.wav"

    # Connect to OpenAI, prompt ChatGPT to summarize the text
    chat_model = ChatOpenAI() # Must have set API key as env var
    prompt = "Summarize the following text in {} words. {}. The summary must be in {}. Here's the text: {}".format(num_words, style, summary_lang, text) 
    summary = chat_model.predict(prompt)

    # Connect to elevenlabs, generate audio
    audio = generate(text=summary, voice=audio_voice, model="eleven_monolingual_v1", api_key=eleven_api_key)

    # Save audiofile
    if not os.path.exists("audiofiles"):
        os.makedirs("audiofiles")

    # Handling identical file names
    file_suffix = 0
    while os.path.exists("audiofiles/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix)):
        file_suffix +=1

    save(audio, "audiofiles/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix))

    return "audiofiles/summary_in_{}_mins_{}.wav".format(num_minutes, file_suffix)




In [23]:
demo = gr.Interface(fn=podify, 
                    inputs=[
                            gr.Textbox(label="What do you want your podcast to be about?", placeholder="E.g. 'This week's news in AI'"),
                            gr.Textbox(label="Do you want to customize the style of your podcast?", placeholder="E.g. 'Explain this to me like I'm five years old'"),
                            gr.Slider(minimum=0.5, maximum=5, step=0.5, label="Length", info = "Number of minutes of audio summary"),
                            gr.Dropdown(["Adam (US)", "Antoni (IT)", "Arnold (US)", "Bella (US)", "Domi (US)", "Elli (US)", "Josh (SWE)", "Rachel (US)", "Sam (US)"], label="Voice", info="Narrator voice and language of podcast"),
                            ],
                    outputs= ["audio"]
                    )
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7869
Running on public URL: https://9f17fd57e0fbb0d7df.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
